In [ ]:
import tkinter as tk
from tkinter import messagebox,ttk
from tkcalendar import Calendar
import datetime
import mysql.connector
import pyttsx3  # Text-to-speech library
import os
import schedule
import time
from threading import Thread
from winsound import Beep  # Import the Beep function from the winsound module

db_connection = mysql.connector.connect(
    host="localhost",
    user="root",
    password="Salah@1#",
    database="DB"
)
cursor = db_connection.cursor()

def on_resize(event):
    # Debounce resizing event by delaying the execution of the resizing function
    root.after(50, resize_task_manager_label)

def resize_task_manager_label():
    # Update label font size on window resize
    if root.state() == "zoomed":
        # If window is maximized, increase font size
        new_font_size = max(15, min(root.winfo_width() // 20, root.winfo_height() // 10)) + 20
    else:
        new_font_size = max(15, min(root.winfo_width() // 20, root.winfo_height() // 10))+40
    label.config(font=("Helvetica", new_font_size))


def add_task():
    # Function to add a new task to the database
    def add_to_db():
        task_name = entry_task_name.get().strip()
        reminder_date = cal.selection_get().strftime('%Y-%m-%d')
        reminder_time = f"{hour_combobox.get()}:{minute_combobox.get()} {ampm_combobox.get()}"
        
        # Check if task name is empty
        if not task_name:
            messagebox.showerror("Error", "Task name cannot be empty.")
            return

        try:
            # Attempt to parse reminder time with AM/PM format
            parsed_time = datetime.datetime.strptime(reminder_time, '%I:%M %p')
            reminder_time = parsed_time.strftime('%H:%M:%S')
        except ValueError:
            messagebox.showerror("Error", "Invalid reminder time format. Please select a valid time and AM/PM.")
            return

        combined_time = f"{reminder_date} {reminder_time}"
        
        cursor.execute("INSERT INTO TASKS (TASK_NAME, REMINDER) VALUES (%s, %s)", (task_name, combined_time))
        db_connection.commit()

        cursor.execute("INSERT INTO COMPLETION_TASKS (TASK_ID, STATUS) VALUES (%s, 'Pending')", (cursor.lastrowid,))
        db_connection.commit()
        
        messagebox.showinfo("Success", "Task added successfully.")

        popup.destroy()

        # Refresh the task list after adding a new task
        refresh_task_list()  # Call the refresh_task_list function to update the task list

    # Create a pop-up dialog for adding a new task
    popup = tk.Toplevel(root)
    popup.title("Add New Task")

    # Add entry field for task name
    tk.Label(popup, text="Task Name:").pack()
    entry_task_name = tk.Entry(popup)
    entry_task_name.pack()

    # Add calendar for selecting date
    tk.Label(popup, text="Reminder Date:").pack()
    cal = Calendar(popup, selectmode="day")
    cal.pack()

    # Add comboboxes for selecting time and AM/PM
    tk.Label(popup, text="Reminder Time:").pack()

    # Hours
    hour_combobox = ttk.Combobox(popup, values=[f"{i:02d}" for i in range(1, 13)], width=5)
    hour_combobox.current(0)  # Default value
    hour_combobox.pack(side="left")

    # Minutes
    minute_combobox = ttk.Combobox(popup, values=[f"{i:02d}" for i in range(0, 60, 5)], width=5)
    minute_combobox.current(0)  # Default value
    minute_combobox.pack(side="left")

    # AM/PM
    ampm_combobox = ttk.Combobox(popup, values=["AM", "PM"], width=5)
    ampm_combobox.current(0)  # Default value
    ampm_combobox.pack(side="left")

    # Add button to confirm addition of the task
    tk.Button(popup, text="OK", command=add_to_db).pack()

root = tk.Tk()
root.title("Resizable App")
root.configure(bg="grey")  # Set background color to grey

# Create a frame to hold the task manager
task_manager_frame = tk.Frame(root, bg="grey")
task_manager_frame.pack(fill=tk.X)  # Cover full screen horizontally
task_manager_frame.grid_propagate(False)  # Prevent resizing

# Create a label inside the frame for the task manager
label = tk.Label(task_manager_frame, text="Task Manager", bg="violet", fg="black", font=("Helvetica", 30, 'bold'))
label.pack(side=tk.TOP, fill=tk.X, pady=(0, 5))  # Add padding above
label.grid_propagate(False)  # Prevent resizing

# Add an "Add Task" button below the label with some padding
add_task_button = tk.Button(task_manager_frame, text="Add Task", command=add_task, font=("Helvetica", 18, 'bold'), width=30, height=3)
add_task_button.pack(side=tk.LEFT, padx=10)  # Place on the left with padding

def task_history():
    # Function to display task history

    # Create a new window for task history
    history_window = tk.Toplevel(root)
    history_window.title("Task History")
    history_window.configure(bg="black")

    # Calculate window dimensions
    screen_width = root.winfo_screenwidth()
    screen_height = root.winfo_screenheight()

    # Add label for task history
    history_label = tk.Label(history_window, text="Task History", font=("Helvetica", 40), bg="yellow", fg="black", width=30, height=3)
    history_label.pack(fill=tk.X, padx=10)  # Increased font size and adjusted padding

    def back_to_main():
        # Function to go back to the main interface
        history_window.destroy()

    # Add back button
    back_button = tk.Button(history_window, text="Back", command=back_to_main, font=("Helvetica", 18), width=20, height=2)
    back_button.pack(pady=(10, 20))

    # Add canvas for details with scrollbar
    canvas = tk.Canvas(history_window, bg="black", highlightthickness=0)
    canvas.pack(side=tk.LEFT, fill=tk.BOTH, expand=True)

    scrollbar = tk.Scrollbar(history_window, orient=tk.VERTICAL, command=canvas.yview)
    scrollbar.pack(side=tk.RIGHT, fill=tk.Y)

    canvas.configure(yscrollcommand=scrollbar.set)

    # Add frame to hold the table_frame
    table_frame_holder = tk.Frame(canvas, bg="black")
    canvas.create_window((0, 0), window=table_frame_holder, anchor='nw')

    # Add frame for table with border
    table_frame = tk.Frame(table_frame_holder, bg="white", bd=10, relief=tk.SOLID)
    #table_frame.pack(anchor=tk.CENTER)  # Center the table frame
    table_frame.pack(pady=(screen_height // 18, screen_height // 18), padx=(screen_width // 4, screen_width // 4))  # Adjust padding and center the table frame

    
    # Add table headers
    headers = ["Sl. Number", "Task Name", "Task Status"]
    for i, header in enumerate(headers):
        tk.Label(table_frame, text=header, font=("Helvetica", 18), bg="black", fg="white").grid(row=0, column=i, padx=10, pady=5)  # Increased font size

    # Query task history from the database
    cursor.execute("SELECT t.TASK_NAME, c.STATUS FROM TASKS t LEFT JOIN COMPLETION_TASKS c ON t.TASK_ID = c.TASK_ID")
    tasks = cursor.fetchall()

    for i, task in enumerate(tasks, start=1):
        task_name, status = task

        # Add task details to the table, centering them horizontally and vertically
        tk.Label(table_frame, text=i, font=("Helvetica", 18), bg="black", fg="white").grid(row=i, column=0, padx=(50, 10), pady=10, sticky="nsew")  # Increased font size and adjusted padding
        tk.Label(table_frame, text=task_name, font=("Helvetica", 18), bg="black", fg="white").grid(row=i, column=1, padx=(50, 10), pady=10, sticky="nsew")  # Increased font size and adjusted padding
        tk.Label(table_frame, text=status, font=("Helvetica", 18), bg="black", fg="white").grid(row=i, column=2, padx=(50, 10), pady=10, sticky="nsew")  # Increased font size and adjusted padding

    # Configure column weights to center the table within the frame
    for col_index in range(3):
        table_frame.grid_columnconfigure(col_index, weight=1)

    # Update scroll region
    table_frame.update_idletasks()
    canvas.config(scrollregion=canvas.bbox("all"))

    # Bind canvas scrolling
    def on_canvas_configure(event):
        canvas.configure(scrollregion=canvas.bbox("all"))
    canvas.bind("<Configure>", on_canvas_configure)


# Add task history button
task_history_button = tk.Button(task_manager_frame, text="Task History", command=task_history, font=("Helvetica", 18, 'bold'), width=30, height=3)
task_history_button.pack(side=tk.LEFT, padx=20)

def exit_confirmation():
    # Function to display exit confirmation dialog
    confirmation_window = tk.Toplevel(root)
    confirmation_window.title("Exit Confirmation")

    # Add label with confirmation message
    confirmation_message = tk.Label(confirmation_window, text="Are you sure you want to exit this app?", font=("Helvetica", 18))
    confirmation_message.pack(padx=20, pady=20)

    # Add 'Yes' button to exit the application
    yes_button = tk.Button(confirmation_window, text="Yes", font=("Helvetica", 18, 'bold'), width=10, command=root.destroy)
    yes_button.pack(side=tk.LEFT, padx=20, pady=10)

    # Add 'No' button to close the confirmation dialog
    no_button = tk.Button(confirmation_window, text="No", font=("Helvetica", 18, 'bold'), width=10, command=confirmation_window.destroy)
    no_button.pack(side=tk.RIGHT, padx=20, pady=10)

exit_button = tk.Button(task_manager_frame, text="Exit", command=exit_confirmation, font=("Helvetica", 18, 'bold'), width=30, height=3)
exit_button.pack(side=tk.RIGHT, padx=(0, 10))

# Add an empty frame for spacing
tk.Frame(root, height=1, bg="grey").pack()

# Create a canvas for the task list with a scrollbar
canvas = tk.Canvas(root, bg="grey", highlightthickness=0)
canvas.pack(side=tk.LEFT, fill=tk.BOTH, expand=True)

scrollbar = tk.Scrollbar(root, orient=tk.VERTICAL, command=canvas.yview)
scrollbar.pack(side=tk.RIGHT, fill=tk.Y)

canvas.configure(yscrollcommand=scrollbar.set)

# Create a frame to hold the task list
task_list_frame = tk.Frame(canvas, bg="grey")
canvas.create_window((0, 0), window=task_list_frame, anchor=tk.NW)

# Bind canvas scrolling
def on_canvas_configure(event):
    canvas.configure(scrollregion=canvas.bbox("all"))
canvas.bind("<Configure>", on_canvas_configure)


def refresh_task_list():
    # Function to refresh the task list displayed on the screen
    for widget in task_list_frame.winfo_children():
        widget.destroy()

    cursor.execute("SELECT t.TASK_ID, t.TASK_NAME, t.REMINDER, c.STATUS FROM TASKS t LEFT JOIN COMPLETION_TASKS c ON t.TASK_ID = c.TASK_ID")
    tasks = cursor.fetchall()

    # Initialize i to -1 outside the loop
    i = -1

    for i, task in enumerate(tasks):
        task_id, task_name, reminder, status = task
        task_frame = tk.PanedWindow(task_list_frame, orient=tk.HORIZONTAL, bd=2, relief=tk.SOLID)
        task_frame.pack(fill=tk.X, padx=10, pady=10)
        task_frame.config(bg="black")  # Set background to black for the entire panedwindow

        # Add checkbox
        check_var = tk.IntVar(value=1 if status == 'Completed' else 0)
        check_button = tk.Checkbutton(task_frame, variable=check_var, command=lambda id=task_id, var=check_var: mark_completed(id, var), bg="black")  # Set background to black
        check_button.pack(side=tk.LEFT)

        # Add task name with increased font size and tab space before remainder time
        tk.Label(task_frame, text=task_name + "\t\t", font=("Helvetica", 14), bg="black", fg="white").pack(side=tk.LEFT,padx=50)  # Set background to black and foreground to white

        # Add remainder time with increased font size
        tk.Label(task_frame, text=str(reminder) + "\t\t", font=("Helvetica", 12), bg="black", fg="white").pack(side=tk.LEFT)  # Set background to black and foreground to white

        # Add delete button
        delete_button = tk.Button(task_frame, text="Delete", command=lambda id=task_id: confirm_delete(id))
        delete_button.pack(side=tk.RIGHT, padx=(0, 500))  # Position the delete button to the right within the panedwindow

    # Check if the loop ran at least once before configuring row span
    if i >= 0:
        task_list_frame.grid_rowconfigure(i, weight=1)

def confirm_delete(task_id):
    # Function to open a confirmation dialog for task deletion
    confirmation = messagebox.askyesno("Confirm Deletion", "Are you sure you want to delete this task?")
    if confirmation:
        delete_task(task_id)

def delete_task(task_id):
    # Function to delete the task from the database and refresh the task list
    try:
        cursor.execute("DELETE FROM TASKS WHERE TASK_ID = %s", (task_id,))
        db_connection.commit()
        refresh_task_list()  # Refresh the task list after deletion
    except mysql.connector.Error as e:
        print("Error:", e)
        messagebox.showerror("Error", "An error occurred while deleting the task.")

# Call refresh_task_list() to display the initial task list
refresh_task_list()

def mark_completed(task_id, check_var):
    status = 'Completed' if check_var.get() == 1 else 'Pending'
    try:
        if status == 'Completed':
            cursor.execute("UPDATE COMPLETION_TASKS SET STATUS = 'Completed' WHERE TASK_ID = %s",
                           (task_id,))
        db_connection.commit()

    except mysql.connector.Error as e:
        print("Error:", e)
        messagebox.showerror("Error", "An error occurred while updating the task status.")



root.bind("<Configure>", on_resize)  # Bind the resize event to the function

# Refresh the task list
refresh_task_list()
'''
def check_reminders():
    while True:
        # Get the current time
        current_time = datetime.datetime.now()

        # Query for reminders that match the current time
        cursor.execute("""
            SELECT t.TASK_NAME, t.REMINDER, c.STATUS
            FROM TASKS t
            LEFT JOIN COMPLETION_TASKS c ON t.TASK_ID = c.TASK_ID
            WHERE t.REMINDER <= %s AND c.STATUS = 'Pending'
            """, (current_time,))
        reminders = cursor.fetchall()

        # Check if any reminders found
        if reminders:
            for task_name, reminder_time, status in reminders:
                # Calculate the time difference between the reminder time and the current time
                time_difference = (reminder_time - current_time).total_seconds()

                # Check if the reminder time is within a certain threshold (e.g., 1 minute)
                if abs(time_difference) < 60:
                    # Play the beep sound
                    Beep(800, 500)

                    # Mark the reminder as completed
                    db_connection.commit()

        # Wait for some time before checking again
        time.sleep(10)  # Adjust interval as needed

# Start the reminder checking thread
reminder_thread = Thread(target=check_reminders)
reminder_thread.daemon = True  # Set as daemon to avoid blocking program exit
reminder_thread.start()'''

# Initialize pyttsx3
engine = pyttsx3.init()

# Adjust speech rate and volume if needed
engine.setProperty('rate', 150)  # You can adjust the speech rate (words per minute)
engine.setProperty('volume', 1)   # You can adjust the volume (0.0 to 1.0)

def check_reminders():
    while True:
        # Get the current time
        current_time = datetime.datetime.now()

        # Query for reminders that match the current time
        cursor.execute("""
            SELECT t.TASK_NAME, t.REMINDER, c.STATUS
            FROM TASKS t
            LEFT JOIN COMPLETION_TASKS c ON t.TASK_ID = c.TASK_ID
            WHERE t.REMINDER <= %s AND c.STATUS = 'Pending'
            """, (current_time,))
        reminders = cursor.fetchall()

        # Check if any reminders found
        if reminders:
            for task_name, reminder_time, status in reminders:
                # Calculate the time difference between the reminder time and the current time
                time_difference = (reminder_time - current_time).total_seconds()

                # Check if the reminder time is within a certain threshold (e.g., 1 minute)
                if abs(time_difference) < 60:
                    # Convert the task name to speech

                    message = f"Hey, it's time for you to: {task_name}"

                    # Convert the message to speech
                    engine.say(message)
                    #engine.say(task_name)
                    engine.runAndWait()  # Wait for speech to finish

                    # Mark the reminder as completed
                    db_connection.commit()

        # Wait for some time before checking again
        time.sleep(10)  # Adjust interval as needed

# Start the reminder checking thread
reminder_thread = Thread(target=check_reminders)
reminder_thread.daemon = True  # Set as daemon to avoid blocking program exit
reminder_thread.start()


root.mainloop()